In [36]:
import numpy as np
from scipy.integrate import odeint
from scipy.integrate import simps
import pandas as pd

In [37]:
M = 10000
a_1 = 10.
b_1 = 1.

mu     = np.random.gamma(a_1, b_1, M)
beta   = np.random.gamma(a_1, b_1, M)
nu     = np.random.gamma(a_1, b_1, M)
sigma  = np.random.gamma(a_1, b_1, M)
kappa  = np.random.gamma(a_1, b_1, M)
K      = np.random.gamma(a_1, b_1, M)

mu_hat     = np.random.gamma(a_1, b_1, M)
beta_hat   = np.random.gamma(a_1, b_1, M)
nu_hat     = np.random.gamma(a_1, b_1, M)
sigma_hat  = np.random.gamma(a_1, b_1, M)
kappa_hat  = np.random.gamma(a_1, b_1, M)
K_hat      = np.random.gamma(a_1, b_1, M)

# Mínimos y máximos
mu_m,mu_M       = 1e-5,0.1
beta_m,beta_M   = 1e-5,0.99
nu_m,nu_M       = 1e-5,0.1
sigma_m,sigma_M = 1e-4,0.9
kappa_m,kappa_M = 1e-6,1e-4
K_m,K_M         = 1e-2,100.

# Normalización de parámetros
mu = (mu_M - mu_m)* ( mu - min(mu) ) / ( max(mu) - min(mu) ) + mu_m
beta = (beta_M - beta_m)* ( beta - min(beta) ) / ( max(beta) - min(beta) ) + beta_m
nu = (nu_M - nu_m)* ( nu - min(nu) ) / ( max(nu) - min(nu) ) + nu_m
sigma = (sigma_M - sigma_m)* ( sigma - min(sigma) ) / ( max(sigma) - min(sigma) ) + sigma_m
kappa = (kappa_M - kappa_m)* ( kappa - min(kappa) ) / ( max(kappa) - min(kappa) ) + kappa_m
K = (K_M - K_m)* ( K - min(K) ) / ( max(K) - min(K) ) + K_m

mu_hat = (mu_M - mu_m)* ( mu_hat - min(mu_hat) ) / ( max(mu_hat) - min(mu_hat) ) + mu_m
beta_hat = (beta_M - beta_m)* ( beta_hat - min(beta_hat) ) / ( max(beta_hat) - min(beta_hat) ) + beta_m
nu_hat = (nu_M - nu_m)* ( nu_hat - min(nu_hat) ) / ( max(nu_hat) - min(nu_hat) ) + nu_m
sigma_hat = (sigma_M - sigma_m)* ( sigma_hat - min(sigma_hat) ) / ( max(sigma_hat) - min(sigma_hat) ) + sigma_m
kappa_hat = (kappa_M - kappa_m)* ( kappa_hat - min(kappa_hat) ) / ( max(kappa_hat) - min(kappa_hat) ) + kappa_m
K_hat = (K_M - K_m)* ( K_hat - min(K_hat) ) / ( max(K_hat) - min(K_hat) ) + K_m

In [38]:
def model (x,t,p):
    """
    p[0]=mu,p[1]=beta,p[2]=nu,p[3]=sigma,p[4]=kappa, p[5]=K
    """
    fx = np.zeros(3)
    xi = 1e-5
    fx[0] = p[0] - ((p[1]*x[1]) + p[2]  + xi  )*x[0]
    fx[1] = ((p[1]*x[1])+ xi ) *x[0] - (p[3] + p[2])*x[1]
    fx[2] = p[3]*x[1] - (p[4] + p[2])*x[2]
    return fx

In [39]:
# En construcción
def R0(p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    S = x[:,0]
    I = x[:,1]
    beta = p[1]
    sigma = p[2]
    nu = p[3]
    f = beta / (sigma + nu)
    return f


# En construcción
def InfectedEndemicP(p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    S = x[:,0]
    I = x[:,1]
    mu    =p[0]
    beta  = p[1]
    sigma = p[2]
    nu    = p[3]
    kappa =p[4]
    f = -(mu*(kappa + nu)*(-beta + nu + sigma)) / (( nu + sigma)*beta*(kappa + nu ) -kappa * sigma )
    return f
    

In [40]:
def prevalenceHIV (p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    S = x[:,0]
    I = x[:,1]
    beta = p[1]
    K = p[5]
    xi = 1e-5
    f = (beta * I + xi) * S
    phi = K * simps(f) * dt
    return phi

In [41]:
def prevalenceAIDS (p):
    x0 = np.array([1,0.00000001,0.00000194])
    time = np.linspace(0.0,34.0,35)
    nn = len(time)
    dt = 1.0/(10.0*nn)
    n_quad = 10.0*nn+1
    t_quad = np.linspace(time[0],time[-1],n_quad)
    x = odeint(model,x0,t_quad,args=(p,))
    I = x[:,1]
    sigma = p[3]
    K = p[5]
    xi = 1e-5
    f = sigma * I
    phi = K * simps(f) * dt
    return phi

In [42]:
p=[1e-5, 1e-5, 1e-5, 1e-4, 1e-6,1e-2]
y_X = prevalenceHIV(p)
print(y_X)

9.999997790349065e-08


/home/kernel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  import sys


In [43]:
# Algorithm to compute Sensitivity Indices

A = np.column_stack((mu,beta, nu, sigma,kappa,K))
B = np.column_stack((mu_hat,beta_hat, nu_hat, sigma_hat,kappa_hat,K_hat))

numcols = len(A[0])
C_list = list()
i=0
while i < numcols:
    B_copy = np.copy(B)
    B_copy[:,i] = A[:,i]
    C_list.append(B_copy)
    i+=1

print("Dimensión de A: " + str(np.shape(A)))
print("Dimensión de B: " + str(np.shape(B)))
print("Dimensión de C: " + str(np.shape(C_list)))

np.savetxt('A.txt', A)
np.savetxt('B.txt', B)

Dimensión de A: (10000, 6)
Dimensión de B: (10000, 6)
Dimensión de C: (6, 10000, 6)


In [44]:
y_A = np.zeros((M,1))
y_B = np.zeros((M,1))
y_C = np.zeros((M,len(C_list)))

for j in range(M):
    y_A[j] = InfectedEndemicP(A[j,:])
    y_B[j] = InfectedEndemicP(B[j,:])

for i in range(len(C_list)):
    for j in range(M):
        y_C[j][i]= InfectedEndemicP(C_list[i][j,:])
        
print(np.shape(y_A))
print(np.shape(y_B))
print(np.shape(y_C))
print(y_A)
print(y_B)
print(y_C)

/home/kernel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


(10000, 1)
(10000, 1)
(10000, 6)
[[ 0.00384495]
 [-0.01379671]
 [-0.55158921]
 ...
 [ 0.0090156 ]
 [ 0.02352211]
 [-0.00518118]]
[[ 0.21079065]
 [ 0.11630519]
 [ 0.04781206]
 ...
 [ 0.03900691]
 [-0.10522662]
 [-0.08182255]]
[[ 0.13929549  0.22193365  0.23228617 -0.00864259  0.2107706   0.21079065]
 [ 0.07168587  0.00985242  0.14318625  0.06860793  0.11629964  0.11630519]
 [ 0.02546054 -1.02148873  0.04280608  0.03772487  0.04781187  0.04781206]
 ...
 [ 0.03196938  0.0702872   0.05437301 -0.02601939  0.03900729  0.03900691]
 [-0.07329507 -0.00246667 -0.10619986 -0.06557409 -0.10522718 -0.10522662]
 [-0.02380968 -0.01506987 -0.06523466 -0.09716839 -0.08182252 -0.08182255]]


In [45]:
y_A_mean = np.sum(y_A)/M
y_B_mean = np.sum(y_B)/M
f_0_sqr = y_A_mean * y_B_mean

print(y_A_mean)
print(y_B_mean)
print(f_0_sqr)

2.3606682750938925
5.180749311638861
12.230030541200382


In [46]:
S = []

sum_yA_sqr = 0
for i in range(M):
    sum_yA_sqr += y_A[i] * y_A[i]
var_Y = (1/M)* sum_yA_sqr - f_0_sqr
    
for i in range(len(C_list)):
    sum_yA_yC = 0    
    for j in range(M):
        sum_yA_yC += y_A[j] * y_C[j][i]       
    var_e_y_q_i = sum_yA_yC/M - f_0_sqr
    print(var_e_y_q_i)
    S.append(var_e_y_q_i/var_Y)

print("sum_yA_sqr: " + str(sum_yA_sqr))
print("var_Y: " + str(var_Y))
print(S)

[-12.48259506]
[16089.18640774]
[-12.39959975]
[-12.39861119]
[-12.4141297]
[-12.41824453]
sum_yA_sqr: [5.88100851e+08]
var_Y: [58797.85502158]
[array([-0.0002123]), array([0.2736356]), array([-0.00021089]), array([-0.00021087]), array([-0.00021113]), array([-0.0002112])]


In [47]:
ST = []

for i in range(len(C_list)):
    sum_yB_yC = 0
    for j in range(M):
        sum_yB_yC += y_B[j] * y_C[j][i]
    var_e_y_q_i = (1/M)* sum_yB_yC - f_0_sqr
    ST.append(1- (var_e_y_q_i/var_Y))
    
print(ST)

[array([-2.70696704]), array([1.00020978]), array([2.39128368]), array([-1.70335765]), array([-2.5338678]), array([-3.54039649])]


In [48]:
indices = np.row_stack((np.transpose(S),np.transpose(ST)))
sobol_indices = pd.DataFrame(indices, index = (['Si', 'STi']), columns= ['mu', 'beta', 'nu', 'sigma', 'kappa', 'K'])
print(sobol_indices)

           mu      beta        nu     sigma     kappa         K
Si  -0.000212  0.273636 -0.000211 -0.000211 -0.000211 -0.000211
STi -2.706967  1.000210  2.391284 -1.703358 -2.533868 -3.540396
